In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import torch
import optuna
from pandas import read_csv
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Tanh
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.optim import lr_scheduler
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import time
import copy


In [4]:
class CSVDataset(Dataset):
    #Constructor for initially loading
    def __init__(self, path):
        df = read_csv(path, header=0)
        self.X = df.values[0:, :-1]
        self.y = df.values[0:, -1]
        self.X = self.X.astype('float32')
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

        print(self.X.shape)
        print(self.y.shape)

    # Get the number of rows in the dataset
    def __len__(self):
        return len(self.X)
    # Get a row at an index
    def __getitem__(self,idx):
        return [self.X[idx], self.y[idx]]

In [5]:
class FraudDetectionMLP(Module):
    def __init__(self, n_inputs, n_layers, layer_sizes, activations):
        super(FraudDetectionMLP, self).__init__()
        assert len(layer_sizes) == n_layers
        assert len(activations) == n_layers
        self.hidden_layers = torch.nn.ModuleList()
        for i in range(n_layers):
            layer = Linear(n_inputs if i == 0 else layer_sizes[i - 1], layer_sizes[i])
            kaiming_uniform_(layer.weight, nonlinearity='relu')
            self.hidden_layers.append(layer)
            if activations[i] == 'relu':
                self.hidden_layers.append(ReLU())
            elif activations[i] == 'sigmoid':
                self.hidden_layers.append(Sigmoid())

        # Output layer
        self.output_layer = Linear(layer_sizes[-1], 1)
        xavier_uniform_(self.output_layer.weight)
        self.output_activation = Sigmoid()

    def forward(self, X):
        for layer in self.hidden_layers:
            X = layer(X)
        output = self.output_layer(X)
        output = self.output_activation(output)
        return output

In [12]:
def objective(trial):
    # search space
    n_layers = trial.suggest_int('n_layers', 1, 5)
    layer_sizes = [trial.suggest_int(f'layer_{i}_size', 10, 250) for i in range(n_layers)]
    activations = [trial.suggest_categorical(f'activation_{i}', ['relu', 'sigmoid']) for i in range(n_layers)]

    # Create model
    model = FraudDetectionMLP(n_inputs=X_train2.shape[1], n_layers=n_layers, layer_sizes=layer_sizes, activations=activations)

    # Define loss function and optimizer
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    # Training loop
    for epoch in range(150):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train2)
        loss = criterion(outputs, y_train2)
        loss.backward()
        optimizer.step()

    # Evaluate on training data
    model.eval()
    with torch.no_grad():
        outputs = model(X_test2)
        predicted_probs = outputs.numpy()

    # Calculate AUC
    auc = roc_auc_score(y_test2.numpy(), predicted_probs)

    return auc  # Maximize AUC

In [19]:
dataset = CSVDataset('/content/drive/MyDrive/Colab Notebooks/AFD/merged_train_data.csv')
X_train2, y_train2 = dataset.X, dataset.y

X_train2 = torch.tensor(X_train2, dtype=torch.float32)
y_train2 = torch.tensor(y_train2, dtype=torch.float32)

dataset = CSVDataset('/content/drive/MyDrive/Colab Notebooks/AFD/merged_test_data.csv')
X_test2, y_test2 = dataset.X, dataset.y

X_test2 = torch.tensor(X_test2, dtype=torch.float32)
y_test2 = torch.tensor(y_test2, dtype=torch.float32)

# Hyperparameter optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get best parameters
best_params = study.best_params
print("Best params:", best_params)

# Extract best parameters
best_n_layers = best_params['n_layers']
best_layer_sizes = [best_params[f'layer_{i}_size'] for i in range(best_n_layers)]
best_activations = [best_params[f'activation_{i}'] for i in range(best_n_layers)]

# Train model with best parameters
best_model = FraudDetectionMLP(n_inputs=X_train2.shape[1],
                               n_layers=best_n_layers,
                               layer_sizes=best_layer_sizes,
                               activations=best_activations)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(best_model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(150):
    best_model.train()
    optimizer.zero_grad()
    outputs = best_model(X_train2)
    loss = criterion(outputs, y_train2)
    loss.backward()
    optimizer.step()


# Test model test data
best_model.eval()
with torch.no_grad():
    outputs2 = best_model(X_test2)
    predicted_probs2 = outputs2.numpy()


(1662, 42)
(1662, 1)


[I 2024-02-21 02:42:59,858] A new study created in memory with name: no-name-334ffbfe-625e-4a1a-ae84-0015a3275571


(125314, 42)
(125314, 1)


[I 2024-02-21 02:43:00,432] Trial 0 finished with value: 0.6196197762591803 and parameters: {'n_layers': 4, 'layer_0_size': 44, 'layer_1_size': 25, 'layer_2_size': 42, 'layer_3_size': 210, 'activation_0': 'relu', 'activation_1': 'relu', 'activation_2': 'relu', 'activation_3': 'sigmoid'}. Best is trial 0 with value: 0.6196197762591803.
[I 2024-02-21 02:43:00,789] Trial 1 finished with value: 0.5940362323228545 and parameters: {'n_layers': 2, 'layer_0_size': 134, 'layer_1_size': 42, 'activation_0': 'relu', 'activation_1': 'sigmoid'}. Best is trial 0 with value: 0.6196197762591803.
[I 2024-02-21 02:43:01,705] Trial 2 finished with value: 0.49064679310563875 and parameters: {'n_layers': 4, 'layer_0_size': 86, 'layer_1_size': 163, 'layer_2_size': 131, 'layer_3_size': 30, 'activation_0': 'relu', 'activation_1': 'sigmoid', 'activation_2': 'sigmoid', 'activation_3': 'relu'}. Best is trial 0 with value: 0.6196197762591803.
[I 2024-02-21 02:43:02,885] Trial 3 finished with value: 0.6312566156648

Best params: {'n_layers': 5, 'layer_0_size': 57, 'layer_1_size': 22, 'layer_2_size': 107, 'layer_3_size': 202, 'layer_4_size': 162, 'activation_0': 'relu', 'activation_1': 'relu', 'activation_2': 'relu', 'activation_3': 'sigmoid', 'activation_4': 'sigmoid'}


In [22]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)

# Adjust layout parameters
fig.update_layout(
    width=600,
    height=300,
    margin=dict(l=50, r=50, t=50, b=50),
)

# Adjust font size of axes labels and title
fig.update_xaxes(title_font_size=14)
fig.update_yaxes(title_font_size=14)
fig.update_layout(title_font_size=16)

# Adjust marker size and color
fig.update_traces(marker=dict(size=6, color='red'))

fig.show(config=plotly_config)

In [21]:
from optuna.visualization import plot_param_importances

plotly_config = {"staticPlot": True}

fig = plot_param_importances(study)

# Adjust layout parameters
fig.update_layout(
    width=600,
    height=300,
    margin=dict(l=50, r=50, t=50, b=50),
)

# Adjust font size of axes labels and title
fig.update_xaxes(title_font_size=14)
fig.update_yaxes(title_font_size=14)
fig.update_layout(title_font_size=16)


fig.show(config=plotly_config)